In [72]:
import ipywidgets as widgets
from aocd import submit

ta = widgets.Textarea(
    placeholder='Input data',
    description='Input:',
    disabled=False
)
ta

Textarea(value='', description='Input:', placeholder='Input data')

In [73]:
sum(len(i) == 2 or len(i) == 3 or len(i) == 4 or len(i) == 7 for line in ta.value.splitlines() for i in line.split("|")[1].strip().split())

514

In [140]:
from collections import defaultdict

def g(s):
    return next(iter(s))

o = frozenset.intersection
a = frozenset("abcdefg")

# The true mapping of wires to digits
_n = ("abcefg", "cf", "acdeg", "acdfg", "bdcf", "abdfg", "abdefg", "acf", "abcdefg", "abcdfg")
n = {frozenset(l): str(i) for i, l in enumerate(_n)}

def p(line):
    i, k = line.split("|")
    l = defaultdict(set)
    m = {}
    
    # l[length] is a set containing the frozensets of (scrambled) inputs with that length
    for word in i.strip().split():
        l[len(word)].add(frozenset(word))

    # g(s) is a helper function that just returns the first (only) value in a set

    # g(l[3]) corresponds to the digit 7 and g(l[2]) corresponds to the digit 1
    # "a" is the only wire found in both
    m["a"] = g(g(l[3]) ^ g(l[2]))
    
    # o(*s) finds the intersection of all the sets s
    # "g" and "a" are the only wires found every digit of length 5 or 6
    # so we find the two wires corresponding to "g" and "a", and subtract
    # the wire we already found for "a"
    m["g"] = g(o(*(l[6] | l[5])) ^ {m["a"]})
    
    # g(l[4]) corresponds to the digit 4. a is the frozenset of all the wires,
    # which I could have just used g(l[7]) for.
    # "e", "a", and "g" are the wires missing from the digit 4. Find those
    # and take out our known wires for "a" and "g"
    m["e"] = g(a ^ g(l[4]) ^ {m["a"], m["g"]})
    
    # "d", "a", and "g" are found in every wire of length 5. Find those and
    # take out our known wires for "a" and "g"
    m["d"] = g(o(*l[5]) ^ {m["a"], m["g"]})
    
    # "c" is the only wire in both the digits 6 and 1. We can find which
    # input is 6 by since of the digits of length 5, only 6 contains "e"
    m["c"] = g(next(w for w in l[5] if m["e"] in w) & g(l[2]))
    
    # The digit 1 contains only "c" and "f". Once we find "c", we know "f"
    m["f"] = g(g(l[2]) ^ {m["c"]})
    
    # "b" is the only one we haven't found yet
    m["b"] = g(a ^ set(m.values()))

    # Turn our mapping into a translation dictionary for use with str.translate
    f = {ord(v): ord(k) for k, v in m.items()}

    return int("".join(n[frozenset(w.translate(f))] for w in k.strip().split()))

print(sum(p(l) for l in ta.value.splitlines()))

1012272


In [143]:
def p_permutate(perms, mapping, line):
    i, k = map(str.split, line.split("|"))
    for perm in perms:
        for w in i:
            if not mapping[sum(map(perm.get, w))]:
                break
        else:
            return int("".join(mapping[sum(map(perm.get, w))] for w in k))

bits = (1, 2, 4, 8, 16, 32, 64, 128)
_n = ("abcefg", "cf", "acdeg", "acdfg", "bdcf", "abdfg", "abdefg", "acf", "abcdefg", "abcdfg")

mapping = [0] * 256
for i in range(10):
    mapping[sum(bits[ord(c)-97] for c in _n[i])] = str(i)

perms = tuple(map(lambda p: dict(zip(p, bits)), permutations("abcdefg")))
print(sum(p_permutate(perms, mapping, line) for line in ta.value.splitlines()))

1012272
